In [30]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential

from keras.layers import Dense, Softmax, Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import pandas as pd

from keras.preprocessing.text import Tokenizer

In [32]:

df_master = pd.read_csv("Example_CSV/imdb_master.csv", encoding='latin-1', index_col = 0)
df_master['sentiment']=df_master['label'].apply(lambda x : 0 if x == 'neg' else 1)


In [33]:



X_train = df_master.loc[:49999,'review'].values
y_train = df_master.loc[:49999,'sentiment'].values  #.apply(lambda x : 0 if x == 'neg' else 1)
X_test = df_master.loc[50000:,'review'].values
y_test = df_master.loc[50000:,'sentiment'].values   #.apply(lambda x : 0 if x == 'neg' else 1)



In [34]:


tokenizer = Tokenizer()
total_reviews = X_train + X_test
tokenizer.fit_on_texts(total_reviews)

max_length = max([len(s.split()) for s in total_reviews])

vocab_size = len(tokenizer.word_index) + 1


X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

X_train_pad = sequence.pad_sequences(X_train_tokens, padding='post', maxlen=max_length)
X_test_pad = sequence.pad_sequences(X_test_tokens, padding='post', maxlen=max_length)

#X_train_pad.shape


In [35]:
top_words = 80000

embedding_vecor_length = 100
model = Sequential()
model.add(Embedding(vocab_size, embedding_vecor_length, input_length = max_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=3, batch_size=64,verbose=2)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2665, 100)         17893900  
_________________________________________________________________
dropout_3 (Dropout)          (None, 2665, 100)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 17,974,401
Trainable params: 17,974,401
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=3, batch_size=500,verbose=2)

Train on 50000 samples, validate on 50000 samples
Epoch 1/3
 - 7222s - loss: 0.6932 - acc: 0.5017 - val_loss: 0.6940 - val_acc: 0.0000e+00
Epoch 2/3


In [12]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test_pad)

y_pred

#print ('Accuracy Score :',accuracy_score(y_test, y_pred) )

array([[1.7464161e-05],
       [1.7464161e-05],
       [1.7464161e-05],
       ...,
       [1.7464161e-05],
       [1.7464161e-05],
       [1.7455768e-05]], dtype=float32)

In [26]:
#twt = ['Meetings: Because none of us is as dumb as all of us.']
twt = ['The movie was good to see.']
#vectorizing the tweet by the pre-fitted tokenizer instance
#twt = tokenizer.texts_to_sequences(twt)

X_test_twt = tokenizer.texts_to_sequences(twt)

X_test_twt_pad = sequence.pad_sequences(X_test_twt, padding='post', maxlen = max_length)

#y_pred = model.predict(X_test_pad)

#y_pred

#padding the tweet to have exactly the same shape as `embedding_2` input
#twt = sequence.pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(X_test_twt_pad)
sentiment = model.predict(X_test_twt_pad, batch_size =1,verbose = 2)[0]

sentiment

[[ 1 14 13 ...  0  0  0]]


array([1.7455768e-05], dtype=float32)

In [27]:
if(numpy.argmax(sentiment) == 0):
    print("negative")
elif (numpy.argmax(sentiment) == 1):
    print("positive")

negative
